In [22]:
import pandas as pd
import numpy as np
import os

In [23]:
# Sample data

DEFAULT_DATE_RANGE = ('2023-01-01', '2025-01-01')

def generate_timeseries_df():
    start, end = DEFAULT_DATE_RANGE
    idx = pd.date_range(start=start, end=end, freq="MS") # freq month start
    n_points = len(idx)
    rng = np.random.default_rng(np.random.randint(1, 99))
    base = rng.normal(loc=5500, scale=1500, size=n_points)
    trend = np.linspace(0, 2000, n_points)
    values = np.maximum(base + trend, 1000).astype(float)
    values = np.minimum(values, 10000).round(2)
    df = pd.DataFrame({"value": values})
    df.insert(0, "date", idx)
    return df
    
def generate_multiline_df():
    start, end = DEFAULT_DATE_RANGE
    idx = pd.date_range(start=start, end=end, freq="MS")

    rng = np.random.default_rng(np.random.randint(1, 99))
    series_names = ["North", "South", "East", "West"]
    frames: list[pd.DataFrame] = []
    for i, name in enumerate(series_names):
        base = 700_000 + i * 120_000
        # Alternate trend direction for some lines
        if i % 2 == 0:
            trend_slope = 400_000 + i * 50_000
            trend = np.linspace(0, trend_slope, len(idx))
        else:
            trend_slope = 400_000 + i * 50_000
            # Down then up: negative slope for first half, positive for second half
            half = len(idx) // 2
            trend = np.concatenate([
                np.linspace(0, -trend_slope, half),
                np.linspace(-trend_slope, trend_slope, len(idx) - half)
            ])
        noise = rng.normal(loc=0, scale=60_000, size=len(idx))
        values = np.maximum(base + trend + noise, 0).round(2)
        frames.append(pd.DataFrame({"date": idx, "series": name, "value": values}))

    df = pd.concat(frames, ignore_index=True)
    return df

In [24]:
bar = generate_timeseries_df()
single_line = generate_timeseries_df()
multi_line = generate_multiline_df()

# Bar chart

In [25]:
bar.head()

,date,value
0,2023-01-01,4813.20
1,2023-02-01,4649.05
2,2023-03-01,5869.60
3,2023-04-01,7015.85
4,2023-05-01,5610.59


# Single line chart

In [26]:
single_line.head()

,date,value
0,2023-01-01,6763.39
1,2023-02-01,1119.17
2,2023-03-01,5209.13
3,2023-04-01,7924.83
4,2023-05-01,3967.39


# Multi line chart

In [27]:
multi_line.head()

,date,series,value
0,2023-01-01,North,676487.81
1,2023-02-01,North,719661.89
2,2023-03-01,North,723421.92
3,2023-04-01,North,739921.84
4,2023-05-01,North,770423.62


In [28]:
# Export the final dfs needed for the charts
os.makedirs('output', exist_ok=True)
bar.to_csv('output/bar.csv', index=False)
single_line.to_csv('output/single_line.csv', index=False)
multi_line.to_csv('output/multi_line.csv', index=False)
